<a href="https://colab.research.google.com/github/samconde/GenerativeSongTitlesAndMusic/blob/main/MusicGeneration/Americana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Magenta Genre Experiment!

Post-Training the Magenta models in an attempt to create more specific models by musical genre.

First, install Magenta:



In [ ]:
#@title Install Magneta & Setup Environment
#@test {"output": "ignore"}

!pip install magenta

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

     |████████████████████████████████| 1.4MB 7.5MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 1.5MB 23.2MB/s 
     |████████████████████████████████| 3.6MB 52.0MB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
     |████████████████████████████████| 2.3MB 48.4MB/s 
     |████████████████████████████████| 1.6MB 55.0MB/s 
     |████████████████████████████████| 358kB 47.2MB/s 
     |████████████████████████████████| 215kB 56.6MB/s 
     |████████████████████████████████| 256kB 53.1MB/s 
     |████████████████████████████████| 5.6MB 18.6MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 368kB 46.9MB/s 
     |████████████████████████████████| 194kB 50.6MB/s 
     |████████████████████████████████| 706kB 43.2MB/s 
     |████████████████████████████████| 368kB 25.5MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 5.5MB 25.7MB/

/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


In [ ]:
#@title Download Genre MIDI Collection

!gdown --id 1gKlIJnnlcTffNZRDjKRtIw0b2fQ-Fe_t

!unzip /content/Americana.zip

Streaming output truncated to the last 5000 lines.
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55tsall.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55tshanghai.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55tsobh.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55ttloah.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55wahthmo.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55wawc.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/55wihwah.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56al.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56bmmh.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56cbcwv.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56ctwosotwr.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56dng.mid  
  inflating: AMERICANA_FOLK_www.pdmusic.org_MIDIRip/1800s/56eya.mid  
  inflatin

In [ ]:
#@title Download Magenta's Pre-Trained Model

!gdown --id 1RHLqShh1NTB-_qI1_NCbQMiMggM2i-cv

!tar -xf /content/hierdec-trio_16bar.tar



Downloading...
From: https://drive.google.com/uc?id=1RHLqShh1NTB-_qI1_NCbQMiMggM2i-cv
To: /content/hierdec-trio_16bar.tar
2.77GB [00:32, 85.8MB/s]
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/gdown/cli.py", line 61, in main
^C


In [ ]:
#@title Convert MIDIs
# TFRecord file that will contain NoteSequence protocol buffers.

!convert_dir_to_note_sequences --input_dir=Americana --output_file=/tmp/notesequences.tfrecord --recursive

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Converted MIDI file Americana/wallace/wvw57mpf.mid.
I0218 21:42:37.468412 140315379001216 convert_dir_to_note_sequences.py:153] Converted MIDI file Americana/wallace/wvw57mpf.mid.
INFO:tensorflow:Converted MIDI file Americana/wallace/wvw71tnutt.mid.
I0218 21:42:37.506828 140315379001216 convert_dir_to_note_sequences.py:153] Converted MIDI file Americana/wallace/wvw71tnutt.mid.
INFO:tensorflow:Converted MIDI file Americana/wallace/wvw59tbr.mid.
I0218 21:42:37.694633 140315379001216 convert_dir_to_note_sequences.py:153] Converted MIDI file Americana/wallace/wvw59tbr.mid.
INFO:tensorflow:Converted MIDI file Americana/wallace/wvw51am.mid.
I0218 21:42:37.755226 140315379001216 convert_dir_to_note_sequences.py:153] Converted MIDI file Americana/wallace/wvw51am.mid.
INFO:tensorflow:Converted MIDI file Americana/wallace/wvw46sg.mid.
I0218 21:42:37.833014 140315379001216 convert_dir_to_note_sequences.py:153] Converted MIDI file 

In [ ]:
#@title Train the model on new MIDI files

!music_vae_train --config=hierdec-trio_16bar --run_dir=/content/ --mode=train --examples_path=/tmp/notesequences.tfrecord

2021-02-19 05:35:40.215288: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
2021-02-19 05:35:52.575861: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-19 05:35:52.605646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-19 05:35:52.672068: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cu

In [ ]:
#@title Load the pre-trained models.
print(BASE_DIR)
trio_models = {}
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
trio_models['hierdec_trio_16bar'] = TrainedModel(hierdec_trio_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_hierdec.ckpt')

flat_trio_16bar_config = configs.CONFIG_MAP['flat-trio_16bar']
trio_models['baseline_flat_trio_16bar'] = TrainedModel(flat_trio_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_flat.ckpt')

gs://download.magenta.tensorflow.org/models/music_vae/colab2
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Dec

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/trio_16bar_hierdec.ckpt
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, MultiOutCategoricalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 0.0, 'max_beta': 1.0, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorf

In [ ]:
#@title Load the current model (Trio).

trio_models = {}
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
trio_models['hierdec_trio_16bar'] = TrainedModel(hierdec_trio_16bar_config, batch_size=4, checkpoint_dir_or_path='/content/train')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Restoring parameters from /content/train/model.ckpt-80128


In [ ]:
#@title Generate 4 samples from the selected model prior.
trio_sample_model = "hierdec_trio_16bar" #@param ["hierdec_trio_16bar", "baseline_flat_trio_16bar"]
temperature = 1.1 #@param {type:"slider", min:0.1, max:1.5, step:0.1}

trio_16_samples = trio_models[trio_sample_model].sample(n=4, length=256, temperature=temperature)
for ns in trio_16_samples:
  play(ns)